In [ ]:
import os
import shutil

LEAD = [1, 2, 3, 4]#, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12] 
TMON = list(range(1, 12+1))
conf = [30]
hidf = [30]
ENN = 5
TEV = 5
EEV = 5
XXX = 64
YYY = 44
ZZZ = 6
CDD = 1.0
HDD = 1.0
EBD = '/scratch/BENGUP/ML/DL_BN/Predictions/Web_V1.0/data/dataset-armor-3d-nrt-monthly_2019_2023_inter_det_ML_MCorrected.nc'
#EBD = '/scratch/BENGUP/ML/DL_BN/Predictions/Web_V1.0/data/dataset-armor-3d-nrt-monthly_2019-2023_anom3_det_t_ML_MCorrected.nc'
HHH = '/scratch/BENGUP/ML/DL_BN/Predictions/Web_V1.0'  # Main directory

for lead in LEAD:
    for tmon in TMON:
        CCC = f'bn_{lead}month_{tmon}_ATL3temp'  # Case Name (Experiment name)

        print(CCC)

        for conf_val in conf:
            for hidf_val in hidf:
                opname = f'C{conf_val}H{hidf_val}'

                # Number of ensemble
                for ens in range(1, ENN + 1):
                    os.makedirs(os.path.join(HHH, 'output', CCC), exist_ok=True)
                    os.makedirs(os.path.join(HHH, 'output', CCC, 'src'), exist_ok=True)
                    os.makedirs(os.path.join(HHH, 'output', CCC, opname), exist_ok=True)
                    os.makedirs(os.path.join(HHH, 'output', CCC, opname, f'EN{ens}'), exist_ok=True)
                    os.makedirs(os.path.join(HHH, 'output', CCC, opname, 'ensmean'), exist_ok=True)

                    src_dir = os.path.join(HHH, 'output', CCC, 'src')
                    shutil.copy2(os.path.join(HHH, 'sample', 'V12.0', 'bn.predictions_.sample'), src_dir)
                    shutil.copy2(os.path.join(HHH, 'sample', 'V12.0', 'bn.ensmean.sample'), src_dir)

                    # Run Predictions
                    with open(os.path.join(src_dir, 'bn.predictions_.sample'), 'r') as file:
                        content = file.read()

                    content = content.replace('convfilter', str(conf_val))
                    content = content.replace('hiddfilter', str(hidf_val))
                    content = content.replace('case', CCC)
                    content = content.replace('samfile', EBD)
                    content = content.replace('opfname', opname)
                    content = content.replace('xxx', str(XXX))
                    content = content.replace('yyy', str(YYY))
                    content = content.replace('TOTSIZ', str(TEV))
                    content = content.replace('zzz', str(ZZZ))
                    content = content.replace('lead_mon', str(lead))
                    content = content.replace('target_mon', str(tmon))
                    content = content.replace('home_directory', HHH)
                    content = content.replace('member', str(ens))

                    with open(os.path.join(src_dir, 'bn.predictions_.py'), 'w') as file:
                        file.write(content)

                    os.system('python ' + os.path.join(src_dir, 'bn.predictions_.py'))

                # compute ensemble mean
                with open(os.path.join(src_dir, 'bn.ensmean.sample'), 'r') as file:
                    content = file.read()

                content = content.replace('opfname', opname)
                content = content.replace('case', CCC)
                content = content.replace('TSTSIZ', str(EEV))
                content = content.replace('convfilter', str(conf_val))
                content = content.replace('home_directory', HHH)
                content = content.replace('numen', str(ENN))

                with open(os.path.join(src_dir, 'bn.ensmean.py'), 'w') as file:
                    file.write(content)

                os.system('python ' + os.path.join(src_dir, 'bn.ensmean.py'))